In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta
from pytz import timezone
import optuna

from hsml.schema import Schema
from hsml.model_schema import ModelSchema

from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline as PPLN
from sklearn.metrics import mean_absolute_error as MAE

import os
import sys
sys.path.append("../src/")
sys.path.append("../")

import config
import data
import data_split
import pipe
import paths
import inference
import featurestoreapi

In [7]:
currentdate = pd.to_datetime(datetime.utcnow()).floor("H") - timedelta(weeks=52)
print(currentdate)

2022-09-26 12:00:00


In [8]:
Features = inference.LoadBatchOfFeaturesFromStore(currentdate)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/104605
Connected. Call `.close()` to terminate connection gracefully.
Fetching data backwards from 2022-08-29 12:00:00 to 2022-09-26 11:00:00


2023-09-25 14:02:08,663 INFO: USE `taxidemandprediction_featurestore`
2023-09-25 14:02:09,116 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`numrides` `numrides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxidemandprediction_featurestore`.`ts_hourly_featuregroup_1` `fg0`
WHERE `fg0`.`pickup_hour` >= TIMESTAMP '2022-08-28 12:00:00.000' AND `fg0`.`pickup_hour` < TIMESTAMP '2022-09-27 11:00:00.000'


In [9]:
Features

,rides_672_hours_before,rides_671_hours_before,rides_670_hours_before,rides_669_hours_before,rides_668_hours_before,rides_667_hours_before,rides_666_hours_before,rides_665_hours_before,rides_664_hours_before,rides_663_hours_before,...,rides_8_hours_before,rides_7_hours_before,rides_6_hours_before,rides_5_hours_before,rides_4_hours_before,rides_3_hours_before,rides_2_hours_before,rides_1_hours_before,pickup_hour,pickup_location_id
241,3.0,2.0,1.0,2.0,2.0,7.0,2.0,0.0,0.0,0.0,...,0.0,2.0,2.0,1.0,1.0,0.0,0.0,1.0,2022-09-26 12:00:00,1
85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-09-26 12:00:00,2
203,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-09-26 12:00:00,3
190,2.0,0.0,2.0,2.0,4.0,1.0,3.0,2.0,3.0,3.0,...,1.0,0.0,3.0,3.0,7.0,7.0,6.0,1.0,2022-09-26 12:00:00,4
17,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-09-26 12:00:00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,39.0,44.0,36.0,38.0,39.0,33.0,42.0,41.0,23.0,23.0,...,3.0,4.0,5.0,14.0,13.0,21.0,34.0,28.0,2022-09-26 12:00:00,261
69,93.0,83.0,88.0,63.0,50.0,58.0,66.0,68.0,35.0,17.0,...,4.0,16.0,59.0,96.0,126.0,136.0,119.0,90.0,2022-09-26 12:00:00,262
171,112.0,83.0,111.0,93.0,92.0,97.0,108.0,63.0,76.0,70.0,...,12.0,19.0,51.0,89.0,96.0,126.0,114.0,125.0,2022-09-26 12:00:00,263
1,143.0,65.0,92.0,112.0,114.0,124.0,130.0,103.0,68.0,78.0,...,14.0,8.0,16.0,29.0,51.0,50.0,82.0,69.0,2022-09-26 12:00:00,264


In [10]:
Model = inference.LoadModelFromRegistry()
Predictions = inference.GetModelPredictions(Model, Features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/104605
Connected. Call `.close()` to terminate connection gracefully.

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [11]:
Predictions["pickup_hour"] = currentdate
Predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,2.0,2022-09-26 12:00:00
1,2,0.0,2022-09-26 12:00:00
2,3,0.0,2022-09-26 12:00:00
3,4,3.0,2022-09-26 12:00:00
4,5,0.0,2022-09-26 12:00:00
...,...,...,...
257,261,39.0,2022-09-26 12:00:00
258,262,92.0,2022-09-26 12:00:00
259,263,116.0,2022-09-26 12:00:00
260,264,93.0,2022-09-26 12:00:00


In [17]:
#Save the Predictions to the Feature Store for later usage

FeatureGroup = featurestoreapi.GetFeatureStore().get_or_create_feature_group(
    name = config.FeatureGroupModelPredictions,
    version = config.FeatureGroupModelPredictionsVersion,
    description = "Predictions Generated by our Production Model",
    primary_key = ["pickup_location_id", "pickup_hour"],
    event_time = "pickup_hour"
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/104605
Connected. Call `.close()` to terminate connection gracefully.


In [19]:
FeatureGroup.insert(Predictions, write_options = {"wait_for_job":False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/104605/fs/104524/fg/115028


Uploading Dataframe: 0.00% |          | Rows 0/262 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/104605/jobs/named/model_predictions_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fe8af1e1d80>, None)